In [1]:
import torch
import torch.nn as nn

from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionInpaintPipeline
from diffusers import utils
from diffusers import UNet2DConditionModel
from diffusers import AudioLDMPipeline

import csv
import IPython.display as ipd

import matplotlib.pyplot as plt
import PIL
from io import BytesIO
from tqdm import tqdm
import skimage
import numpy as np
from scipy.io.wavfile import write
import argparse, os, sys, datetime, glob, importlib, csv

from PIL import Image

sys.path.append(os.getcwd())
# import cv2
import torchaudio
import torchaudio.transforms as transforms
import time
from encodec import EncodecModel
from encodec.utils import convert_audio
import torch.nn.functional as F
from transformers import AutoTokenizer, T5EncoderModel,T5TokenizerFast

%matplotlib inline

In [2]:
model_path = "/u/li19/data_folder/model_cache/audio_journey_128_ddim_2"
pipe = StableDiffusionPipeline.from_pretrained(
    model_path,
    device_map=None, 
    safety_checker=None,
    low_cpu_mem_usage=False)
device = "cuda"
pipe.to(device)


/u/li19/data_folder/anaconda3/envs/diffuse/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.15.1",
  "feature_extractor": [
    "transformers",
    "CLIPFeatureExtractor"
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "T5TokenizerFast"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
print(pipe.unet)

UNet2DConditionModel(
  (conv_in): Conv2d(128, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (encoder_hid_proj): Linear(in_features=1024, out_features=768, bias=True)
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_fe

In [4]:
# conf = "/u/li19/data_folder/testing/audio_journey_t5_short_strong_mixup/checkpoint-74000/unet/"

# conf = "/u/li19/data_folder/testing/audio_journey_t5_short_time_mixup/checkpoint-44000/unet/"
# conf = "/u/li19/data_folder/testing/audio_journey_t5_short_temp_no_mixup/checkpoint-68000/unet/"
conf = "/u/li19/data_folder/testing/audio_journey_t5_formant_test/checkpoint-0/unet/"
# conf = "/u/li19/data_folder/testing/audio_journey_t5_shorter_mixup/checkpoint-10000/unet/"
new_unet = UNet2DConditionModel.from_pretrained(conf, device_map=None, 
    safety_checker=None, low_cpu_mem_usage=False, ignore_mismatched_sizes=True)
new_unet.to(device)
pipe.unet = new_unet

Some weights of UNet2DConditionModel were not initialized from the model checkpoint at /u/li19/data_folder/testing/audio_journey_t5_formant_test/checkpoint-0/unet/ and are newly initialized: ['class_embedding.weight', 'class_embedding.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of UNet2DConditionModel were not initialized from the model checkpoint at /u/li19/data_folder/testing/audio_journey_t5_formant_test/checkpoint-0/unet/ and are newly initialized because the shapes did not match:
- down_blocks.0.resnets.0.time_emb_proj.weight: found shape torch.Size([320, 1280]) in the checkpoint and torch.Size([320, 2560]) in the model instantiated
- down_blocks.0.resnets.1.time_emb_proj.weight: found shape torch.Size([320, 1280]) in the checkpoint and torch.Size([320, 2560]) in the model instantiated
- down_blocks.1.resnets.0.time_emb_proj.weight: found shape torch.Size([640, 1280]) in the checkpoint and torch

In [5]:
print(pipe.unet.config.projection_class_embeddings_input_dim)
print(pipe.unet.config.class_embed_type)
print(pipe.unet.config.class_embeddings_concat)
print(pipe.unet.config.out_channels)
print(pipe.unet.config.time_embedding_type)

600
simple_projection
True
128
positional


In [6]:
time_embed_dim = pipe.unet.config.block_out_channels[0] * 4
print(time_embed_dim)

1280


In [30]:
class_embedding = nn.Linear(pipe.unet.config.projection_class_embeddings_input_dim, time_embed_dim)


In [ ]:
class_labels = torch.rand(3, 200)
class_labels = torch.flatten(class_labels)
print(class_labels.shape)
class_emb = class_embedding(class_labels)
# emb = torch.cat([emb, class_emb], dim=-1)


In [ ]:
print(class_embedding.bias.dtype)

In [33]:
print(class_emb.shape)

torch.Size([3, 1280])


In [7]:
pipe.save_pretrained("/u/li19/data_folder/model_cache/audio_journey_t5_formant_v_concat")

In [13]:
formants = torch.rand(10, 3, 200)


In [14]:
flattened_tensor = formants.view(formants.size(0), -1)


In [15]:
print(flattened_tensor.shape)  # Should output torch.Size([10, 600])


torch.Size([10, 600])
